# HW № 2: Taggers' Battle!

**Организатор битвы: Андрей Якубой, БКЛ212**

In [ ]:
!pip3 install nltk

In [ ]:
from nltk import download
from nltk.tokenize import WordPunctTokenizer, sent_tokenize


download("punkt")
tokenizer = WordPunctTokenizer()

## 1. Корпус и Золотой Стандарт

Никакое соревнование не обходится без полосы препятствий с коварными
ловушками - их расстановкой и займёмся!

Наша коллекция представляет собой набор несвязных предложений из двух
источников. Первый - НКРЯ. Я искал примеры с аномальными формами,
искажёнными формами, фамилии, заканчивающиеся на *ой*, *ых* и другие
интересные штуки.

А вот второй источник...

[Лурк! Великий и ужасный](https://lurkmore.pro/)

Во-первых, я всегда мечтал поработать компьютерными методами с луркоязом и
посмотреть, что из этого выйдет, а во-вторых - здесь, может быть, и нет
особо аномальных форм или обманок, но зато огромное количество "неизвестных"
 слов и искажённых (намеренно) написаний.

Ниже я разобью исходную коллекцию на предложения и покажу некоторые цитаты и
 из примеров из НКРЯ, и из Уютненького.

In [ ]:
# Не воспроизводить заново!

input_file = "hw2_collection.txt"
output_file = "hw2_collection_splitted.txt"

with open(input_file, "r", encoding="utf-8") as f:
    text = f.read()

sentences = sent_tokenize(text, language='russian')

with open(output_file, "w", encoding="utf-8") as f:
    for sentence in sentences:
        f.write(sentence + "\n")

In [ ]:
len(sentences)

Тут, честно говоря, пришлось немного поправить ручками: неверно разбилась
название "Что? Где? Когда?".

Итак, что же интересного есть в нашем корпусе?

Лурк: "неизвестные" слова (хотя вполне регулярные), а также намеренно
искажённые в стиле луркояза написания слов.

> «Кто хочет стать миллионером?» — **педерача**, в которой актеры гонятся за **3кк** рублей (**1кк** евро/долларов/стерлингов, в странах с меньшей валютой **5кк**), отвечая на вопросы и разводя зрителя на отправку смс. Является излюбленной едой фотожабберов и развлекательных **педерач**.

Здесь, кроме неконвенциональных слов, стоит обратить внимание на популярную
сейчас в Интернете запись числительных: 3кк=3000000, 5кк=5000000 и ТП.
Вопрос, как это размечать и разбирать, дискуссионный, но лично я склоняюсь к
 варианту разметить как числительные - заодно и посмотрим, что умеют эти
 ваши парсеры!

> В 2019 году в одном из выпусков должны были играть звёзды «Что? Где? Когда?» **Друзь** и **Сиднев**. Сыграли они так себе, забрав всего 200 тысяч **рублёв**.

Здесь фамилии, плюс иненормативная форма р.п. у *рубль* - интересно,
додумаются, что
это не имя собственное?

> Поскольку начитка аудиокниги кажется простой задачей, многим **надмозгам**
 > приходит в голову мысль: «а набубню-ка я в микрофон от гарнитуры
 > очередной фанфик **гаррипоттера** и буду продавать/раздавать @
 > прославлюсь». В итоге получается нечто мерзкое, шепелявое, шумящее, сравнимое по качеству только с русским **фандабом**, удаляемое самим **надмозгом** после первого же прослушивания. Но иногда запись расползается по интернетам и взрывает мозг неосторожно послушавшим её **аудиокниголюбам**.

Тут без комментариев!

В случае с примерами из НКРЯ я целенаправленно искал необычные словоформы,
которые потенциально могут представлять трудности для парсеров. Здесь и
слегка устаревшие формы глаголов, и сложные слова с первой неизменяемой
частью, и формы типа *ваще* и с местоимениями на *-нить*... Есть и
сокращения, и фамилии на *-ых*, и "ловушки" со словами, оканчивающимися на
*-ята*.

> Здесь и победа может славы не **принесть**, А пораженье — заслужить аплодисменты.

> Хитрющие улыбки, уместное серьезное выражение лица, циничные фразочки и убойнейший махач – сценаристы молодцы, создали убойного **вампир-хантера**, но и Уэсли сделал все, как надо.

> А ты знаешь про эту поэму **что-нить** и **ваще** что надо про нее знать?

> Еще в древности буддисты и даосы знали, что пустота есть сущность бытия "**шуньята** — это **татхата** "

> — На других огородах **арбузята** родятся мелкие, кислые

## Тегсет

Буду использовать UD, потому что (сюрприз!) это универсальный формат! Многие
 языконезависимые парсеры, включая UDPipe, отдают данные именно в этом
 формате, а для остальных парсеров нетрудно и сконвертировать. Постараюсь
 использовать полный набор частиречных тегов, ничего не упрощая - ну а
 дальше посмотрим, что отдадут парсеры.

По формату: это будет сильно урезанное подобие CONLL-U. В качестве
комментария (#) я запишу предложение целиком, а ниже буду делать разметку с
полями `token, lemma, pos` для каждого слова. Пустая строка - разделитель
предложений.

Зачем лемма? А потому что в прошлом году я писал курсовую по методам
лемматизации, и мне интересно ещё посмотреть на лучший лемматизатор, а не
только на лучший парсер! А мотивации такой у меня больше никогда не будет. :(

Токенизируем коллекцию и подготовим к разметке.

In [ ]:
# Тоже не воспроизводить заново!

input_file = "hw2_collection_splitted.txt"
output_file = "hw2_corpus.txt"

with open(input_file, "r", encoding="utf-8") as in_file, open(output_file, "w", encoding="utf-8") as out_file:
    sentences = in_file.read().splitlines()
    for sentence in sentences:
        out_file.write("# " + sentence + "\n")
        tokens = tokenizer.tokenize(sentence)
        for token in tokens:
            if token.isalnum():
                out_file.write(token.lower() + "\t\t" + "\n")
        out_file.write("\n")

Ну всё, я погнал это размечать, не переключайтесь!

...

...

...

*звуки долгой и мучительной разметки ручками*

**Я ЭТО СДЕЛАЛ!!!**

Из сложного: имена собственные, написанные с маленькой буквой (в т.ч. и с
искажениями - Лурк): *нульч*, *гаррипоттер*. Сомневаюсь, что хоть один
анализатор сумеет это распознать, но я всё же разметил их как `PROPN`. Есть
много случаев омонимии в служебных частях речи: *и* как союз и как частица,
например.

## 2. Представляем участников

Natasha - специально заточенная под русский язык штука, решающая разные
задачи, в том числе и морфологический анализ. Славится своей
производительностью и качеством, разве что чуть-чуть худшим, чем у огромной
неповоротливой DeepPavlov. Одна беда: в основе - новостной датасет, а не
что-то нормальное...

Stanza - мощный NLP-пайплайн с нейросетевой архитектурой. Модели обучены на
датасетах UD.

UDPipe2 - примерно такой же пайплайн, как и Stanza (только лемматизатор не
нейросетевой), но интересен тем, что при обучении моделей вычисляет
контекстуализированные эмбеддинги для токенов с помощью BERT. В теории это
значит, что мы можем брать более продвинутую информацию о семантике
(=контексте), что должно помочь в анализе. Кажется, так больше пока никто не
 делал, они первые.

SpaCy - тоже многоязычный и многофункциональный пайплайн, а также очень
удобный фреймворк для тренировки самых разных моделей на любой вкус и под
любую задачу. Русский датасет тоже новостной.

То есть у нас есть четыре нейросетевых архитектуры, одна из них заточена
специально под русский язык, три другие - многоязычные, и одна из них пытается
 в семантику и продвинутый анализ контекста. Что ж, посмотрим, посмотрим!

In [ ]:
!pip3 install natasha stanza spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.1/880.1 kB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 103.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=d29eb5c89b8e58a3cdc86cb3b04716a7b68257bf7cf1bd980abb6aaa89acea4e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=c0fa563e6c8c02c974ca744f538c8c7d507edf705fa4ebb0695102afd7f621d5
  Stored 

In [ ]:
!python3 -m spacy download ru_core_news_lg

2023-10-05 19:11:26.490715: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-05 19:11:28.366464: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-05 19:11:32.177890: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-05 19:11:32.178422: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

Для Stanza и UDPipe2 будем использовать модели, обученные на корпусе "Тайга" - он размечен руками. Хотя в конце мы попробуем потестить и "СинТагРус",
 который больше по объёму, но размечен автоматически.

In [ ]:
import requests

import stanza
import spacy

from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

stanza_nlp = stanza.Pipeline("ru", package="Taiga")
spacy_nlp = spacy.load("ru_core_news_lg")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package        |
------------------------------
| tokenize  | taiga          |
| pos       | taiga_charlm   |
| lemma     | taiga_nocharlm |
| depparse  | taiga_charlm   |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


С UDPipe2 всё интереснее: его очень геморно разворачивать вручную, но есть
онлайновый REST API, чем мы и воспользуемся, написав функцию, принимающую
текст и название модели и возвращающая размеченные данные в CONLL-U.

> Важно: к сожалению, сейчас функция будет работать только при включённом VPN (если только вы не в Колабе) - с российских IP запросы не идут, сервер возвращает 451. :(

In [ ]:
def udpipe2(text, model):
    url = "http://lindat.mff.cuni.cz/services/udpipe/api/process"
    params = {
        "tokenizer": "TOKENIZER",
        "tagger": "TAGGER",
        "parser": "PARSER",
        "data": text,
        "model": model
    }
    response = requests.get(url, params=params)
    return response.json()["result"]

Так уж получилось, что все четыре инструмента делают разметку в формате UD, что
 избавляет нас от проблем с по-разному размечеными частями речи.

## 3. Делаем автоматическую разметку нашей коллекции

> "— Голдстейн.
>
> — Да, мистер Старк.
>
> — Дай мощный бит, под который я буду бить лучшего друга".

© *Железный человек 2*

> Важно: перед просмотром третьей и четвёртой частей тетрадки категорически рекомендуется включить на полную громкость [этот трек](https://www.youtube.com/watch?v=rY0WxgSXdEE), иначе ощущения могут быть не те!

In [ ]:
input_file = "hw2_collection_splitted.txt"

with open(input_file, "r", encoding="utf-8") as inp_file:
    sentences = inp_file.read().splitlines()

### Первый ход: Stanza

In [ ]:
with open("hw2_stanza.txt", "w", encoding="utf-8") as out_file:
    for sentence in sentences:
        out_file.write("# " + sentence + "\n")
        doc = stanza_nlp(sentence)
        for sent in doc.sentences:
            for token in sent.words:
                word = token.text.lower()
                if word.isalnum() or (len(word) > 1 and "-" in word):
                    lemma = token.lemma
                    pos = token.upos
                    out_file.write(f"{word}\t{lemma}\t{pos}\n")
        out_file.write("\n")

print("Раунд!")

Раунд!


### Ход 2: Natasha

In [ ]:
with open("hw2_natasha.txt", "w", encoding="utf-8") as out_file:
    for sentence in sentences:
        out_file.write("# " + sentence + "\n")
        doc = Doc(sentence)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for token in doc.tokens:
            token.lemmatize(morph_vocab)
            word = token.text.lower()
            if word.isalnum() or (len(word) > 1 and "-" in word):
                lemma = token.lemma
                pos = token.pos
                out_file.write(f"{word}\t{lemma}\t{pos}\n")
        out_file.write("\n")

print("Раунд!")

Раунд!


### ход 3: UDPipe2! Встречаем!

In [ ]:
model = "russian-taiga-ud-2.12-230717"

with open("hw2_udpipe2.txt", "w", encoding="utf-8") as out_file:
    for sentence in sentences:
        out_file.write("# " + sentence + "\n")
        conllu_output = udpipe2(sentence, model)
        for line in conllu_output.split('\n'):
            if not line or line.startswith("#"):
                continue
            fields = line.split("\t")
            if len(fields) > 4:
                word = fields[1].lower()
                if word.isalnum() or (len(word) > 1 and "-" in word):
                    lemma = fields[2]
                    pos = fields[3]
                    out_file.write(f"{word}\t{lemma}\t{pos}\n")
        out_file.write("\n")

print("Раунд!")

Раунд!


### И наконец: SpaCy!

In [ ]:
with open("hw2_spacy.txt", "w", encoding="utf-8") as out_file:
    for sentence in sentences:
        out_file.write("# " + sentence + "\n")
        doc = spacy_nlp(sentence)
        for token in doc:
            word = token.text.lower()
            if word.isalnum() or (len(word) > 1 and "-" in word):
                lemma = token.lemma_
                pos = token.pos_
                out_file.write(f"{word}\t{lemma}\t{pos}\n")
        out_file.write("\n")

print("Раунд!")

Раунд!


## 4. Определяем победителя

Теперь, имея результаты всех участников, мы можем посчитать accuracy и
сказать, кто таки круче!

Проблема в том, что не все токенизировали предложения так, как это
соответствует моей картине мира. Например, в Золотом Стандарте слова с
частицей *-ка* разделяются на два токена, а слова типа *вампир-хантер* не
разделяются. Есть и траблы с токенизацией числительных типа *3кк* и *5кк*. В
 связи с этим, при подсчёте accuracy мы будем штрафовать участника не только
  за неправильный POS-тег, но и за неправильные токены.

Ниже - функция, которая берёт файл с Золотым Стандартом и файл с
автоматической разметкой и оценивает accuracy.

In [ ]:
from collections import defaultdict

def evaluate_pos_accuracy(gold_path, auto_path):
    gold_dict = defaultdict(list)
    auto_dict = defaultdict(list)
    correct_count = 0
    total_count = 0

    with open(gold_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.startswith("#") and line.strip():
                token, lemma, pos = line.strip().split("\t")
                gold_dict[token].append(pos)

    with open(auto_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.startswith("#") and line.strip():
                token, lemma, pos = line.strip().split("\t")
                auto_dict[token].append(pos)

    errors = []
    for token, gold_pos_list in gold_dict.items():
        if token in auto_dict:
            auto_pos_list = auto_dict[token]

            for g_pos, a_pos in zip(gold_pos_list, auto_pos_list):
                total_count += 1
                if g_pos == a_pos:
                    correct_count += 1
                else:
                    errors.append({
                        "token": token,
                        "g_pos": g_pos,
                        "a_pos": a_pos
                    })

            total_count += abs(len(gold_pos_list) - len(auto_pos_list))

        else:
            total_count += len(gold_pos_list)
            errors.append({
                "token": token,
                "a_pos": None
            })

    accuracy = correct_count / total_count if total_count > 0 else 0
    return {"accuracy": accuracy, "errors": errors}

Идея за этой функцией вот какая: если токен из золотого стандарта есть в
словаре, мы ориентируемся на порядок этого конкретного токена, проверяя тег
у каждого вхождения (например, токены *и* бывают как союзами, так и
частицами). Если токена в авторазметке вовсе нет, просто не засчитываем без
вопросов и переходим дальше.

In [ ]:
# Stanza

stanza_eval = evaluate_pos_accuracy("hw2_corpus.txt", "hw2_stanza.txt")
stanza_eval["accuracy"]

0.9115281501340483

In [ ]:
# Natasha

natasha_eval = evaluate_pos_accuracy("hw2_corpus.txt", "hw2_natasha.txt")
natasha_eval["accuracy"]

0.8262032085561497

In [ ]:
# UDPipe2

udpipe2_eval = evaluate_pos_accuracy("hw2_corpus.txt", "hw2_udpipe2.txt")
udpipe2_eval["accuracy"]

0.9037433155080213

In [ ]:
# SpaCy

spacy_eval = evaluate_pos_accuracy("hw2_corpus.txt", "hw2_spacy.txt")
spacy_eval["accuracy"]

0.8586666666666667

И у нас, кажется, определился победитель!

Встречаем: Stanza!!! Кто бы мог подумать! Я-то ставил на UDPipe2!

Но давайте посмотрим, где ошиблись наши участники!

In [ ]:
stanza_eval["errors"]

[{'token': '5кк', 'g_pos': 'NUM', 'a_pos': 'PROPN'},
 {'token': '2019', 'g_pos': 'NUM', 'a_pos': 'ADJ'},
 {'token': 'что', 'g_pos': 'PRON', 'a_pos': 'SCONJ'},
 {'token': 'где', 'g_pos': 'PRON', 'a_pos': 'ADV'},
 {'token': 'когда', 'g_pos': 'PRON', 'a_pos': 'ADV'},
 {'token': 'всего', 'g_pos': 'PART', 'a_pos': 'ADV'},
 {'token': 'жж', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'это', 'g_pos': 'PART', 'a_pos': 'PRON'},
 {'token': 'это', 'g_pos': 'DET', 'a_pos': 'PART'},
 {'token': 'нульча', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'нульч', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'своим', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'похуй', 'g_pos': 'ADV', 'a_pos': 'VERB'},
 {'token': 'похуй', 'g_pos': 'ADV', 'a_pos': 'VERB'},
 {'token': 'торрентсру', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'свои', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'как', 'g_pos': 'SCONJ', 'a_pos': 'ADV'},
 {'token': 'многим', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'ка', 'g_pos': 

In [ ]:
natasha_eval["errors"]

[{'token': '3кк', 'a_pos': None},
 {'token': '1кк', 'a_pos': None},
 {'token': '5кк', 'a_pos': None},
 {'token': 'и', 'g_pos': 'PART', 'a_pos': 'CCONJ'},
 {'token': 'и', 'g_pos': 'PART', 'a_pos': 'CCONJ'},
 {'token': 'и', 'g_pos': 'PART', 'a_pos': 'CCONJ'},
 {'token': 'и', 'g_pos': 'PART', 'a_pos': 'CCONJ'},
 {'token': '2019', 'g_pos': 'NUM', 'a_pos': 'ADJ'},
 {'token': 'одном', 'g_pos': 'DET', 'a_pos': 'NUM'},
 {'token': 'что', 'g_pos': 'PRON', 'a_pos': 'SCONJ'},
 {'token': 'где', 'g_pos': 'PRON', 'a_pos': 'ADV'},
 {'token': 'когда', 'g_pos': 'PRON', 'a_pos': 'ADV'},
 {'token': 'сыграли', 'g_pos': 'VERB', 'a_pos': 'ADJ'},
 {'token': 'amdmitry', 'g_pos': 'X', 'a_pos': 'PROPN'},
 {'token': 'amdmitry', 'g_pos': 'X', 'a_pos': 'PROPN'},
 {'token': 'это', 'g_pos': 'PART', 'a_pos': 'PRON'},
 {'token': 'это', 'g_pos': 'DET', 'a_pos': 'PART'},
 {'token': 'вин', 'g_pos': 'NOUN', 'a_pos': 'ADJ'},
 {'token': 'асечка', 'g_pos': 'NOUN', 'a_pos': 'ADJ'},
 {'token': 'нульча', 'g_pos': 'PROPN', 'a_pos

In [ ]:
udpipe2_eval["errors"]

[{'token': '5кк', 'a_pos': None},
 {'token': '2019', 'g_pos': 'NUM', 'a_pos': 'ADJ'},
 {'token': 'что', 'g_pos': 'PRON', 'a_pos': 'SCONJ'},
 {'token': 'где', 'g_pos': 'PRON', 'a_pos': 'ADV'},
 {'token': 'когда', 'g_pos': 'PRON', 'a_pos': 'ADV'},
 {'token': 'друзь', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'всего', 'g_pos': 'PART', 'a_pos': 'ADV'},
 {'token': 'это', 'g_pos': 'PART', 'a_pos': 'PRON'},
 {'token': 'это', 'g_pos': 'DET', 'a_pos': 'PART'},
 {'token': 'нульча', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'нульч', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'своим', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'похуй', 'g_pos': 'ADV', 'a_pos': 'VERB'},
 {'token': 'похуй', 'g_pos': 'ADV', 'a_pos': 'VERB'},
 {'token': 'торрентсру', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'свои', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'макак', 'g_pos': 'NOUN', 'a_pos': 'PROPN'},
 {'token': 'многим', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'набубню', 'a_pos': None},
 

In [ ]:
spacy_eval["errors"]

[{'token': 'и', 'g_pos': 'PART', 'a_pos': 'CCONJ'},
 {'token': 'и', 'g_pos': 'PART', 'a_pos': 'CCONJ'},
 {'token': 'и', 'g_pos': 'PART', 'a_pos': 'CCONJ'},
 {'token': 'и', 'g_pos': 'PART', 'a_pos': 'CCONJ'},
 {'token': '2019', 'g_pos': 'NUM', 'a_pos': 'ADJ'},
 {'token': 'одном', 'g_pos': 'DET', 'a_pos': 'NUM'},
 {'token': 'где', 'g_pos': 'PRON', 'a_pos': 'ADV'},
 {'token': 'когда', 'g_pos': 'PRON', 'a_pos': 'ADV'},
 {'token': 'amdmitry', 'g_pos': 'X', 'a_pos': 'PROPN'},
 {'token': 'amdmitry', 'g_pos': 'X', 'a_pos': 'PROPN'},
 {'token': 'это', 'g_pos': 'PART', 'a_pos': 'PRON'},
 {'token': 'это', 'g_pos': 'DET', 'a_pos': 'PART'},
 {'token': 'нульча', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'нульч', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'своим', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'однако', 'g_pos': 'CCONJ', 'a_pos': 'ADV'},
 {'token': 'всем', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'всем', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'похуй', 'g_pos': 'ADV',

Ну чё я могу сказать...

Из спорных случаев: абсолютно все парсеры решили, что *2019* в *2019 год* -
это прилагательное, что по крайней мере доказывает, что они правда пытаются
парсить синтаксис. Если мы принимаем, что существуют штуки типа
числительных-существительных и числительных-прилагательных (как с
местоимениями), да, пожалуй, такой вариант имеет право на существование.

Никнейм *amdmitry*. В ЗС помечен как `X` (иностранное слово), Stanza и UD
определяют также, а вот остальные предпочитают считать, что это имя
собственное. Тоже не могу не согласиться с допустимостью такого варианта.

Предикатив *надо* - два парсера распознают как наречие, а два - как глагол.
В UD нет тега для предикатива, поэтому в стандарте размечено как глагол (ориентация на корпус "Тайга").
Мне кажется, сближать предикативы с глаголами не стоит хотя бы из-за
совершенно разной морфологии этих слов, набора грамматических категорий - но
 как раз два лучших парсера - Stanza и UDPipe2 - считают, что таки глагол...

Наблюдаются большие сложности с разрешением неоднозначности в случае с
*это*: местоимение, частица или DET (в русском языке как будто нет такой
части речи, но в UD, в частности в Тайге, размечено) - тут у парсеров
полнейший хаос.

Не очень понятно, почему массово всеми помечаются как `det` слова *свой*,
*его*. Мне сложно представить контекст, в котором это было бы законно, если
только не считать определителями любое употребление этих местоимений.

Парадоксально, но только SpaCy с его ужасно наивным токенизатором смог
правильно выделить числительные *3кк*, *1кк* и *5кк* - он правильно их
разметил. Остальные пытались отделить буквы от чисел, в результате чего *к*
определялось как существительное. Ну и причуда от победителя: для Stanza
выдал "имя собственное" для *5кк*, хотя остальные разметил нормально.

К сожалению, никто не смог правильно определить частицу *-ка*: даже если
токенизация и проходила правильно, то частица определялась либо как
междометие (Stanza), либо как существительное (SpaCy".

Напоследок только скажу ещё, что ни один из парсеров, увы, не справился с
одним экспрессивным наречием, приводить которое здесь не буду, но оно есть в
 выдаче. А жаль (

Вообще становится очевидным, что обучение на новостных датасетах, чтобы
потом работать с чем-то другим - не очень хорошая идея, что показывает
перформанс SpaCy и Наташи (особенно последней, я вообще был в шоке, что она
отработала даже хуже SpaCy). Есть трудности в определении части речи даже на
 самых элементарных словах: *фразочки*, *упущенное* и другие. Очень плохое
 разрешение неоднозначности: например, *и* как частицу не определили ни разу. Хотя "для индустрии, а не для науки", чё с них с бедных взять при таких-то целях и заявлениях!

В общем, пока рулят тяжёлые модели, обученные на больших синтаксически
аннотированных корпусах. Интересно было бы обучить модели в SpaCy и Natasha
на Тайге и посмотреть, что будет с перформансом.

### БОНУС: модели Stanza и UDPipe2, обученные на СинТагРусе

SynTagRus - ещё один размеченный корпус. Он намного больше, чем "Тайга", но
там много автоматической разметки, т.е. он более "грязный".

У Stanza и UDPipe2 есть модели, обученные на СинТагРусе. Интересно проверить,
что решит - качество
или количество?

In [ ]:
# Stanza

stanza_syntagrus = stanza.Pipeline("ru", package="SynTagRus")

with open("hw2_stanza_syntagrus.txt", "w", encoding="utf-8") as out_file:
    for sentence in sentences:
        out_file.write("# " + sentence + "\n")
        doc = stanza_syntagrus(sentence)
        for sent in doc.sentences:
            for token in sent.words:
                word = token.text.lower()
                if word.isalnum() or (len(word) > 1 and "-" in word):
                    lemma = token.lemma
                    pos = token.upos
                    out_file.write(f"{word}\t{lemma}\t{pos}\n")
        out_file.write("\n")

print("Раунд!")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


Раунд!


In [ ]:
# UDPipe2

model = "russian-syntagrus-ud-2.12-230717"

with open("hw2_udpipe2_syntagrus.txt", "w", encoding="utf-8") as out_file:
    for sentence in sentences:
        out_file.write("# " + sentence + "\n")
        conllu_output = udpipe2(sentence, model)
        for line in conllu_output.split('\n'):
            if not line or line.startswith("#"):
                continue
            fields = line.split("\t")
            if len(fields) > 4:
                word = fields[1].lower()
                if word.isalnum() or (len(word) > 1 and "-" in word):
                    lemma = fields[2]
                    pos = fields[3]
                    out_file.write(f"{word}\t{lemma}\t{pos}\n")
        out_file.write("\n")

print("Раунд!")

Раунд!


In [ ]:
stanza_syntagrus_eval = evaluate_pos_accuracy("hw2_corpus.txt", "hw2_stanza_syntagrus.txt")
stanza_syntagrus_eval["accuracy"]

0.8847184986595175

In [ ]:
udpipe2_syntagrus_eval = evaluate_pos_accuracy("hw2_corpus.txt", "hw2_udpipe2_syntagrus.txt")
udpipe2_syntagrus_eval["accuracy"]

0.8873994638069705

Ого, прикол! У нас при таком раскладе меняется лидер!

In [ ]:
stanza_syntagrus_eval["errors"]

[{'token': 'долларов', 'a_pos': None},
 {'token': '5кк', 'g_pos': 'NUM', 'a_pos': 'NOUN'},
 {'token': '2019', 'g_pos': 'NUM', 'a_pos': 'ADJ'},
 {'token': 'одном', 'g_pos': 'DET', 'a_pos': 'NUM'},
 {'token': 'что', 'g_pos': 'PRON', 'a_pos': 'SCONJ'},
 {'token': 'где', 'g_pos': 'PRON', 'a_pos': 'ADV'},
 {'token': 'когда', 'g_pos': 'PRON', 'a_pos': 'SCONJ'},
 {'token': 'друзь', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'всего', 'g_pos': 'PART', 'a_pos': 'ADV'},
 {'token': 'amdmitry', 'g_pos': 'X', 'a_pos': 'PROPN'},
 {'token': 'amdmitry', 'g_pos': 'X', 'a_pos': 'PROPN'},
 {'token': 'это', 'g_pos': 'PART', 'a_pos': 'PRON'},
 {'token': 'это', 'g_pos': 'DET', 'a_pos': 'PRON'},
 {'token': 'нульча', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'нульч', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'своим', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'однако', 'g_pos': 'CCONJ', 'a_pos': 'ADV'},
 {'token': 'этим', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'похуй', 'g_pos': 'ADV', 'a_

In [ ]:
udpipe2_syntagrus_eval["errors"]

[{'token': 'кто', 'a_pos': None},
 {'token': 'евро', 'a_pos': None},
 {'token': '5кк', 'g_pos': 'NUM', 'a_pos': 'NOUN'},
 {'token': '2019', 'g_pos': 'NUM', 'a_pos': 'ADJ'},
 {'token': 'одном', 'g_pos': 'DET', 'a_pos': 'NUM'},
 {'token': 'что', 'g_pos': 'PRON', 'a_pos': 'SCONJ'},
 {'token': 'где', 'g_pos': 'PRON', 'a_pos': 'ADV'},
 {'token': 'когда', 'a_pos': None},
 {'token': 'amdmitry', 'g_pos': 'X', 'a_pos': 'PROPN'},
 {'token': 'amdmitry', 'g_pos': 'X', 'a_pos': 'PROPN'},
 {'token': 'это', 'g_pos': 'PART', 'a_pos': 'PRON'},
 {'token': 'это', 'g_pos': 'DET', 'a_pos': 'PRON'},
 {'token': 'нульча', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'нульч', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'своим', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'однако', 'g_pos': 'CCONJ', 'a_pos': 'ADV'},
 {'token': 'похуй', 'g_pos': 'ADV', 'a_pos': 'VERB'},
 {'token': 'торрентсру', 'g_pos': 'PROPN', 'a_pos': 'NOUN'},
 {'token': 'свои', 'g_pos': 'PRON', 'a_pos': 'DET'},
 {'token': 'как', 'g_pos

Что ж, качество, всё-таки, тут оказалось важнее количество. Интересно, что
моделька UDPipe2 в случае с SynTagRus обрабатываеть чуточку лучше, чем
аналогичная у Stanza - хотя разница прям минимальная.

## 5. Mission 2: Chunking

Победитель - модель Stanza, обученная на корпусе "Тайга" - проходит в
следующий раунд, и тут уже всё серьёзно (нет)!

Будем пытаться улучшать алгоритм из первого ДЗ (а это вообще возможно?)
путём добавления в словарь с уникальными словами и эмоджи ещё и биграмм. Но
не просто биграмм, а таких, которые соответствуют некоторым паттернам.

Здесь нам и поможет наш победитель: мы его эксплуатируем в универсальном
чанкере, который принимает на вход текст и шаблоны, и возвращает все
биграммы, которые этим шаблонам соответствуют. Итак, вот
штука!

In [ ]:
def chunker(text, templates, nlp):
    words = []
    doc = nlp(text)
    for template in templates:
        part1, part2 = template.split("+")
        for sentence in doc.sentences:
            for i in range(len(sentence.words) - 1):
                first_word = sentence.words[i]
                second_word = sentence.words[i + 1]
                if part1.startswith("|"):
                    check1 = first_word.upos == part1[1:]
                else:
                    check1 = first_word.text.lower() == part1.lower()
                if part2.startswith("|"):
                    check2 = second_word.upos == part2[1:]
                else:
                    check2 = second_word.text.lower() == part2.lower()
                if check1 and check2:
                    words.append(f"{first_word.text.lower()} {second_word.text.lower()}")
    return words

Здесь принимаем следующее: части шаблона разделяются знаком плюса, а "труба"
 (|) ставится перед элементом шаблона, который обозначает частиречный тег.

### Дублируем алгоритм из первого задания

Уже без смешных комментариев - если хочется их вспомнить, можно посмотреть
соседний файл в том же репо.

Единственное, что мы изменим - функцию предобработки во втором задании:
ранее лемматизация и токенизация происходила с помощью Mystem, а мы изменим
на Stanza.

In [ ]:
!pip3 install emot scikit-learn tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.2 MB/s eta 0:00:00


In [ ]:
from collections import Counter
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
from nltk import download
from nltk.corpus import stopwords

In [ ]:
np.random.seed(42)
tqdm.pandas()

download("stopwords")
sw = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
reviews = pd.read_csv("app_store_reviews.csv")

In [ ]:
positive_reviews = reviews[reviews["is_positive"] == 1]
negative_reviews = reviews[reviews["is_positive"] == 0]
positive_sample = positive_reviews.sample(n=250, random_state=42)
negative_sample = negative_reviews.sample(n=250, random_state=42)

reviews = reviews.drop(positive_sample.index)
reviews = reviews.drop(negative_sample.index)

eval_positive_reviews = reviews[reviews["is_positive"] == 1]
eval_negative_reviews = reviews[reviews["is_positive"] == 0]
eval_positive_sample = eval_positive_reviews.sample(n=50, random_state=42)
eval_negative_sample = eval_negative_reviews.sample(n=50, random_state=42)
eval_sample = pd.concat([eval_positive_sample, eval_negative_sample])

In [ ]:
def preprocess_text(text):
    doc = stanza_nlp(text)
    lemmas = []
    for sentence in doc.sentences:
        for word in sentence.words:
            lemmas.append(word.lemma)
    cleaned_lemmas = [lemma.lower() for lemma in lemmas if lemma and lemma.isalpha()]
    return cleaned_lemmas

> Даня! Если ты правда это перезапускаешь, возьми Колаб и GPU - сэкономь время!

In [ ]:
positive_sample["words"] = positive_sample["review_text"].progress_apply(
    preprocess_text
)
negative_sample["words"] = negative_sample["review_text"].progress_apply(
    preprocess_text
)
eval_sample["words"] = eval_sample["review_text"].progress_apply(
    preprocess_text
)

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
positive_counter = Counter()
negative_counter = Counter()

for words_list in positive_sample['words']:
    positive_counter.update(words_list)

for words_list in negative_sample['words']:
    negative_counter.update(words_list)

positive_frequency_threshold = 2
negative_frequency_threshold = 4

positive_words = set()
negative_words = set()

for word, count in positive_counter.items():
    if count >= positive_frequency_threshold:
        positive_words.add(word)

for word, count in negative_counter.items():
    if count >= negative_frequency_threshold:
        negative_words.add(word)

common_words = positive_words.intersection(negative_words)
positive_words -= common_words
negative_words -= common_words
positive_words -= sw
negative_words -= sw

In [ ]:
from emot.core import emot

emot_obj = emot()

In [ ]:
def extract_emoticons(text):
    emojis = emot_obj.emoji(text)["value"]
    text_emoticons = emot_obj.emoticons(text)["value"]
    return emojis + text_emoticons

In [ ]:
positive_sample["emoticons"] = positive_sample["review_text"].apply(
    extract_emoticons
)
negative_sample["emoticons"] = negative_sample["review_text"].apply(
    extract_emoticons
)
eval_sample["emoticons"] = eval_sample["review_text"].apply(
    extract_emoticons
)

In [ ]:
positive_emoticons = set()
negative_emoticons = set()

for emot_list in positive_sample["emoticons"]:
    positive_emoticons.update(emot_list)

for emot_list in negative_sample["emoticons"]:
    negative_emoticons.update(emot_list)

common_emoticons = positive_emoticons.intersection(negative_emoticons)
positive_emoticons -= common_emoticons
negative_emoticons -= common_emoticons

In [ ]:
class AppStoreClassifier(BaseEstimator):
    def __init__(self, positive_words, negative_words, positive_emoticons, negative_emoticons):
        self.positive_words = positive_words
        self.negative_words = negative_words
        self.positive_emoticons = positive_emoticons
        self.negative_emoticons = negative_emoticons

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        predictions = []
        for tokens in X:
            positive_word_count = sum(1 for token in tokens if token in positive_words)
            negative_word_count = sum(1 for token in tokens if token in negative_words)
            positive_emoticon_count = sum(1 for token in tokens if token in self.positive_emoticons)
            negative_emoticon_count = sum(1 for token in tokens if token in self.negative_emoticons)
            positive_count = positive_word_count + (2 * positive_emoticon_count)
            negative_count = negative_word_count + (2 * negative_emoticon_count)
            if positive_count > negative_count:
                predictions.append(1)
            else:
                predictions.append(0)
        return predictions

In [ ]:
X = eval_sample["words"] + eval_sample["emoticons"]
y = eval_sample["is_positive"]

clf = AppStoreClassifier(positive_words, negative_words, positive_emoticons, negative_emoticons)
y_pred = clf.predict(X)

In [ ]:
accuracy_score(y, y_pred)

0.58

Класс! Мы просто поменяли теггер с Mystem на Stanza и получили +1% к качеству! (в прошлой ДЗ было 0.57, предобработка была на Mystem)

### Пытаемся улучшить бедняжку

Добавим в наши словарики биграммы, уникальные для положительных и
отрицательных отзывов. Возьмём такие шаблоны, которые, как правило, хорошо
показывают тональность отзыва. Они будут вот такими:

* `не+|ADJ` - тут очевидно;
* `не+|ADV` - поймает штуки типа *не очень*;
* `|ADV+|ADJ` - *очень плохой* VS. *очень хороший*;
* `|ADJ+|NOUN` - *хорошее приложение* VS. *отвратительное приложение*;
* `не+|VERB` - *не грузит* VS. *не лагает*;
* `|ADV+|ADV` - *очень плохо* VS. *очень хорошо*.

По классике жанра: сделаем новую колонку в датафрейме, куда и положим списки
 с биграммами.

In [ ]:
templates = [
    "не+|ADJ",
    "не+|ADV",
    "|ADV+|ADJ",
    "|ADJ+|NOUN",
    "не+|VERB",
    "|ADV+|ADV"
]

positive_sample["bigrams"] = positive_sample["review_text"].progress_apply(
    chunker, args=(templates, stanza_nlp)
)
negative_sample["bigrams"] = negative_sample["review_text"].progress_apply(
    chunker, args=(templates, stanza_nlp)
)
eval_sample["bigrams"] = eval_sample["review_text"].progress_apply(
    chunker, args=(templates, stanza_nlp)
)

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

#### Извлекаем уникальные

In [ ]:
positive_bigram_counter = Counter()
negative_bigram_counter = Counter()

for bigram_list in positive_sample['bigrams']:
    positive_bigram_counter.update(bigram_list)

for bigram_list in negative_sample['bigrams']:
    negative_bigram_counter.update(bigram_list)

positive_frequency_threshold = 1
negative_frequency_threshold = 1

positive_bigrams = set()
negative_bigrams = set()

for bigram, count in positive_bigram_counter.items():
    if count >= positive_frequency_threshold:
        positive_bigrams.add(bigram)

for bigram, count in negative_bigram_counter.items():
    if count >= negative_frequency_threshold:
        negative_bigrams.add(bigram)

common_bigrams = positive_bigrams.intersection(negative_bigrams)
positive_bigrams -= common_bigrams
negative_bigrams -= common_bigrams

In [ ]:
len(positive_bigrams)

166

In [ ]:
len(negative_bigrams)

615

In [ ]:
len(common_bigrams)

26

In [ ]:
positive_bigrams

{'3 день',
 'адекватными админами',
 'бесплатные подарки',
 'блестящий форум',
 'большая просьба',
 'буквально вчера',
 'видно фото',
 'вовлвдмш ггч',
 'вроде нормально',
 'встроеная функция',
 'вы-наше спасение',
 'главное меню',
 'главной странице',
 'данной сети',
 'данной соцсети',
 'доброе утро',
 'добрый вечер',
 'дорогие разрабочики',
 'единственный недостаток',
 'ещё большая',
 'живое общение',
 'известную ошибку',
 'как сильно',
 'как так',
 'классное приложение',
 'конкретном каталоге',
 'крутая приложения',
 'крутая соц-сеть',
 'куда дели',
 'личного списка',
 'лучшее приложение',
 'лучший мессенджер',
 'любимые одноклассники',
 'милые разработчики',
 'милые разрабы',
 'мобильной версии',
 'музыкального видео',
 'намного интереснее',
 'не блокируете',
 'не вернули',
 'не вижу',
 'не виснет',
 'не включать',
 'не восстанавливают',
 'не выдаёт',
 'не глючит',
 'не готова',
 'не грузит',
 'не знали',
 'не значатся',
 'не известную',
 'не как',
 'не корректно',
 'не критично',
 

In [ ]:
negative_bigrams

{'$$аные ремиксы',
 '2018 года',
 'ÿeбищное приложение',
 'абсолютно бесполезную',
 'абсолютно неудобным',
 'активная анкета',
 'альтернативного варианта',
 'банальный видеоролик',
 'бездарные функции',
 'безполезным ширпотребом',
 'безыскусный дизайн',
 'белого мужчины',
 'бесконечное количество',
 'бесплатную музыку',
 'бесплатных подарков',
 'бесплатных стикеров',
 'бесполезную вещь',
 'бесполезные разделы',
 'бессмысленный скрол',
 'более длинные',
 'более комфортные',
 'более неудобным',
 'более удобные',
 'большая стоимость',
 'большей части',
 'большой вопрос',
 'былую славу',
 'былые времена',
 'быстрого доступа',
 'быстрой работой',
 'быстром доступе',
 'важное обновление',
 'великолепное решение',
 'вечной рекламы',
 'вечные лаги',
 'визуальный контент',
 'вкладка видео',
 'внешний вид',
 'вообще бесполезное',
 'вообще никак',
 'вообще отдельная',
 'враждебными высказываниями',
 'все больше',
 'все зашкварнее',
 'все невыносимее',
 'все хуже',
 'всегда худшие',
 'всё более',


In [ ]:
common_bigrams

{'вчерашнего дня',
 'добрый день',
 'крутое приложение',
 'не видно',
 'не возможно',
 'не заходил',
 'не заходит',
 'не знаю',
 'не могу',
 'не может',
 'не нравится',
 'не осталось',
 'не открывается',
 'не открываются',
 'не отображается',
 'не понятно',
 'не работает',
 'не так',
 'не удобно',
 'новое обновление',
 'норм приложение',
 'последнего обновления',
 'соц сеть',
 'соц. сети',
 'старые версии',
 'уважаемые разработчики'}

Мне нравится список, по-моему получилось достаточно релевантно. Да, у нас опять сильный перекос в сторону отрицательных отзывов, но здесь не получается сделать баланс, как со словами - при выставлении порога 2 биграмм уже неприлично мало, а если такой поставить только для негативных отзывов, получится уже другой перекос, в сторону положительных...

#### Обновляем классификатор

In [ ]:
class AppStoreClassifier(BaseEstimator):
    def __init__(self, positive_words, negative_words, positive_emoticons, negative_emoticons, positive_bigrams, negative_bigrams):
        self.positive_words = positive_words
        self.negative_words = negative_words
        self.positive_emoticons = positive_emoticons
        self.negative_emoticons = negative_emoticons
        self.positive_bigrams = positive_bigrams
        self.negative_bigrams = negative_bigrams

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        predictions = []
        for review in X:
            positive_word_count = sum(1 for token in review["tokens"] if token in self.positive_words)
            negative_word_count = sum(1 for token in review["tokens"] if token in self.negative_words)
            positive_emoticon_count = sum(1 for token in review["tokens"] if token in self.positive_emoticons)
            negative_emoticon_count = sum(1 for token in review["tokens"] if token in self.negative_emoticons)
            positive_bigram_count = sum(1 for bigram in review["bigrams"] if bigram in self.positive_bigrams)
            negative_bigram_count = sum(1 for bigram in review["bigrams"] if bigram in self.negative_bigrams)
            positive_count = positive_word_count + (2 * positive_emoticon_count) + 2 * positive_bigram_count
            negative_count = negative_word_count + (2 * negative_emoticon_count) + 2 * negative_bigram_count
            if positive_count > negative_count:
                predictions.append(1)
            else:
                predictions.append(0)
        return predictions

Здесь мы немного модифицировали структуру *X*: это больше не список, а
словарь - потому что биграммы смешивать с униграммами нельзя.

#### Момент истины!

In [ ]:
X_tokens = eval_sample["words"] + eval_sample["emoticons"]
X_bigrams = eval_sample["bigrams"]
X = [{"tokens": tokens, "bigrams": bigrams} for tokens, bigrams in zip(X_tokens, X_bigrams)]
y = eval_sample["is_positive"]

clf = AppStoreClassifier(positive_words, negative_words, positive_emoticons , negative_emoticons, positive_bigrams, negative_bigrams)
y_pred = clf.predict(X)

In [ ]:
accuracy_score(y, y_pred)


0.61

Ура, +3 процента! Больше нуля, то есть того, что я ожидал!

Итак, к концу первой домашки, где мы использовали исключительно лемматизированные униграммы и эмоджи, мы получили 57%. Здесь мы подняли 1% за счёт использование более лучшего (я надеюсь) лемматизатора и токенизатора - вероятно, списки униграмм стали более релевантными. Ещё 3% мы подняли засчёт учёта биграмм. Причина в том, что анализ биграмм позволяет анализировать слова, которые могли не попасть в список униграмм из-за слишком редкой встречаемости, но делать это более осмысленно засчёт сочетания с другими словами, которые многое говорят о тональности отзыва.

> Значит, парсер Stanza молодец и успешно прошёл второй этап! А третьего организаторы пока не придумали, потому что не было аванса, Олег...

FINISH!
